# South Atlantic BeachBirds

This indicator was used in the 2022 base blueprint.

In 2022, it was not used in all zonation areas where the source data overlaps, so we added a step to clip it down to the area where it was used in zonation. We will revisit the exent in 2023

This indicator was originally made for the South Atlantic Blueprint using zonation to combine beach bird species data.

Created by Amy Keister, last run by Amy Keister on 2 August 2022. It took 1 minute to run

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBPrep.gdb"

In [4]:
# define final indicator outputs
#SEout = r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBeachBirds2023.tif"

In [5]:
# define outputs to be used in zonation
AMOY= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\AMOY.tif"
WIPL= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\WIPL.tif"
PIPL= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\PIPL.tif"
LETE= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\LETE.tif"
zMask= r"D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\zMaskNo0.tif"

In [6]:
# define rasters used for cell size, extent, and snapping
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [7]:
# define inputs
birds= r"F:\GIS_DATA\SpeciesAndHabitats\BeachBirds\2_0\Beach_birds_core_area_blp0_no_edge_removal\Beach_birds_core_area_no_edge_removal\outputs\Beach_birds_core_area_no_edge_removal.CAZ_.rank.compressed.tif"
#SubR= r"E:\WORKING\BaseBlueprint\2022\1_SubregionsAndExtent\BaseBlueprintSubRgn.shp"
VH= r"F:\GIS_DATA\SpeciesAndHabitats\BeachBirds\VonHolle2011\GIS\SHOREBIRD_REGIONAL_POLY_WITH_RANK\SHOREBIRD_REGIONAL_POLY_WITH_RANK.shp"
PIPLin= r"F:\GIS_DATA\SpeciesAndHabitats\BeachBirds\2_0\AlbersEA\Bird_species\PIPL_index_2km_90m.img"
LETEin= r"F:\GIS_DATA\SpeciesAndHabitats\BeachBirds\2_0\AlbersEA\Bird_species\LETE_rank_90m.img"
SAraster= r"F:\GIS_DATA\SALCC_BLUEPRINT\Version_2021\Blueprint_2021_Data_Download_20220204\1_ExtentLayers\SA2021extent.tif"
Rextent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

### Start analysis

In [8]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [9]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\5_Indicators_Tier2_UnClipped\SouthAtlanticBeach\SouthAtlanticBPrep.gdb


### Bring in zonation results from Brad

In [10]:
# take zonation results times 100
#out_raster = arcpy.sa.Times(birds, 100); out_raster.save("birds")

In [11]:
# reclassify
#out_raster = arcpy.sa.Reclassify("birds", "VALUE", "0 20 1;20 40 2;40 60 3;60 80 4;80 100 5", "DATA"); out_raster.save("indicator1")

### Make rasters from VonHolle data

In [12]:
# make a copy of VonHolle Shorebird regional poly for edits
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.CopyFeatures(VH, "VH", '', None, None, None)

In [13]:
# buffer VonHolle data to make an estimate of extent
# buffer by 50 km to try to retain nearshore hardbottom
# 50 km was chosen by a quick visual exploration, this number might need to be tweaked
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer(VH, "VHbuff50", "50 Kilometers", "FULL", "ROUND", "None", None, "PLANAR")

In [14]:
# dissolve buffers (when I try to dissolve while doing the buffer it takes forever)
arcpy.management.Dissolve("VHbuff50", "VHbuff50Dis", "BUFF_DIST", None, "MULTI_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\5_Indicators_Tier2_UnClipped\\SouthAtlanticBeach\\SouthAtlanticBPrep.gdb\\VHbuff50Dis'>

In [15]:
# add fields for raster conversion
#arcpy.management.AddFields("VH", "WIPL_RANK_NUM SHORT # # # #;AMOY_RANK_NUM SHORT # # # #")


In [16]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 'Extremely High':
        return 6
    elif value == 'Very High':
        return 5
    elif value == 'High':
        return 4
    elif value == 'Moderately High':
        return 3
    elif value == 'Moderate':
        return 2 
    elif value == 'Low':
        return 1
    elif value == 'None':
        return 0 
"""

In [17]:
# add and calculate numeric field based on WIPL_RANK
arcpy.management.CalculateField("VH", "WIPL_RANK_NUM", "Reclass(!WIPL_RANK!)", "PYTHON3", codeblock, "SHORT")

<Result 'VH'>

In [18]:
# add and calculate numeric field based on AMOY_RANK
arcpy.management.CalculateField("VH", "AMOY_RANK_NUM", "Reclass(!AMOY_RANK!)", "PYTHON3", codeblock, "SHORT")

<Result 'VH'>

In [19]:
# convert to raster using the numeric rank for American Oystercatcher
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("VH", "AMOY_RANK_NUM", AMOY, "MAXIMUM_COMBINED_AREA", "NONE", 30)

In [20]:
# convert to raster using the numeric rank for Wilson's Plover
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("VH", "WIPL_RANK_NUM", WIPL, "MAXIMUM_COMBINED_AREA", "NONE", 30)

### Make .tifs from Brad' data on Least Tern and Piping Plover

In [21]:
# reproject least tern raster
#with arcpy.EnvManager(outputCoordinateSystem=sr, mask=SAraster, extent=SAraster, snapRaster=SAraster):
#    arcpy.management.ProjectRaster(LETEin, "LETEp", SAraster)

In [22]:
# resample least tern raster as 30 meter
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.Resample(LETEin, "LETE", "30 30", "NEAREST")

In [23]:
# export as .tif. For some reason when I tried to export as tif at the same time as resample, the extent was wrong
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("LETE", LETE, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [24]:
# resample and export piping plover raster as 30 meter .tif
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.Resample(PIPLin, "PIPL", "30 30", "NEAREST")

In [25]:
# export as .tif. For some reason when I tried to export as tif at the same time as resample, the extent was wrong
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("PIPL", PIPL, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

### Make a mask for zonation

In [26]:
# use cell statistics to combine all the data
out_raster = arcpy.sa.CellStatistics("PIPL.tif;LETE.tif;WIPL.tif;AMOY.tif", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("all")

In [29]:
# reclassify the above raster to make a mask for zonation
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con("all", "1", "", "Value >0"); out_raster.save("zMask")

In [30]:
# clip mask to the 50 km buffer around VonHolle data
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("zMask", "VHbuff50Dis"); out_raster.save("zMask2")

In [31]:
# clip mask to the 50 km buffer around VonHolle data
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=LETEin, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("zMask2", SAraster); out_raster.save(zMask)

In [32]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

194.4865918159485
